In [1]:

import torch
import numpy as np
import random
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import keyboard
import sys
sys.path.append("model") 


In [2]:
# 准备数据
X,y = torch.load('data_fake.pth')  #输入数据




# 数据预处理 标准化数据

# 划分训练集、验证集和测试集
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=20)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=20)

# 将数据转移到 GPU（如果可用

# 检查GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, X_val, X_test = X_train.to(device), X_val.to(device), X_test.to(device)
y_train, y_val, y_test = y_train.to(device), y_val.to(device), y_test.to(device)



In [3]:
device

device(type='cuda')

In [4]:
class LinearResNetBlock(nn.Module):
    def __init__(self):
        super(LinearResNetBlock, self).__init__()
        
        self.hidden_dim = 64
        self.block_layer_nums =3
            
        # Define layers for the function f (MLP)
        self.layers = nn.ModuleList()
        
        for _ in range(self.block_layer_nums - 1):  # -2 because we already added one layer and last layer is already defined
            self.layers.append(nn.Linear(self.hidden_dim,self.hidden_dim ))
        
        # Layer normalization
        self.layernorms = nn.ModuleList()
        for _ in range(self.block_layer_nums - 1):  # -1 because layer normalization is not applied to the last layer
            self.layernorms.append(nn.LayerNorm(self.hidden_dim))
        
    def forward(self, x):
        # Forward pass through the function f (MLP)
        out = x
        for i in range(self.block_layer_nums - 1):  # -1 because last layer is already applied outside the loop
            out = self.layers[i](out)
            out = self.layernorms[i](out)
            out = torch.relu(out)
        
        # Element-wise addition of input x and output of function f(x)
        out = x + out
        
        return out

# 定义残差网络18/34的残差结构，为2个3*3的卷积
class BasicBlock(nn.Module):
    # 判断残差结构中，主分支的卷积核个数是否发生变化，不变则为1
    expansion = 1
    def __init__(self,in_channel,out_channel,stride=1,downsample=None,**kwargs):
        super(BasicBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,
                               kernel_size=3,stride=stride,padding=1,bias=False
        )
        # 使用批量归一化
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,
                               kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample =downsample
    def forward(self,x):
 
        identity = x
        if self.downsample is not None:
            identity=self.downsample(x)  # self当前对象实例，用于访问对象的属性和方法
        out = self.conv1(x)
        out =self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out +=identity
        out = self.relu(out)
        return out
class Bottleneck(nn.Module):
    # expansion是指在每个小残差块内，减少尺度增加维度的倍数
    # 该类输出的通道是输入的四倍
    expansion = 4
    def __init__(self,in_channel,out_channel,strdie=1,downsample=None,
                 groups=1,width_per_group=64):
        super(Bottleneck,self).__init__()
 
        width = int(out_channel*(width_per_group/64.))*groups
 
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=width,kernel_size=1,stride=1,bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2= nn.Conv2d(in_channels=width,out_channels=width,groups=groups
                              ,kernel_size=3,stride=strdie,bias=False,padding=1)
        self.bn2 = nn.BatchNorm2d(width)
 
        self.conv3 = nn.Conv2d(in_channels=width,out_channels=width,groups=groups,
                               kernel_size=3,stride=strdie,bias=False,padding=1)
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansio)
 
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
    def forward(self,x):
        identify = x
        if self.downsample is not None:
            identify = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
 
        out+=identify
        out = self.relu(out)
        return out
    


#定义ResNet类
class ResNet(nn.Module):
    def __init__(self,block_type,blocks_num,linear = False,include_top=False,groups=1,width_per_group=64):
        super(ResNet,self).__init__()
        self.include_top = include_top
        self.linear = linear
        # maxpool的输出通道数为8，残差结构的输入通道为8
        self.origin_chanel = 1
        self.in_channel = 8
        self.groups = groups
        self.width_per_group = width_per_group
        self.conv1 = nn.Conv2d(self.origin_chanel,self.in_channel,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.conv2 = nn.Conv2d(16,1,kernel_size=1)
        # 浅层步长为1，深层的步长为2
        # block：定义了两种残差模块
        # block_num:定义了残差快的个数
        self.layer1 = self._make_layer(block_type,8,blocks_num[0])
        self.layer2 = self._make_layer(block_type, 8, blocks_num[1],stride=2)
        self.layer3 = self._make_layer(block_type, 16, blocks_num[2],stride=2)
        self.layer4 = self._make_layer(block_type, 16, blocks_num[3],stride=2)

        self.fc1 = nn.Linear(64,64)

        if self.linear == True :
            self.linearResnet = LinearResNetBlock()
        # 遍历网络中的每一层
        # 继承nn.Moudle类中的一个方法：self.Moudle(),它会返回该网络中所有的moudles
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                # kaiming正态分布初始化，使得卷积层反向传播的输出的方差都为1
                # fan_in权重是通过线性层隐形确定
                # fan_out：通过创建随机矩阵显式创建权重
                nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
    def _make_layer(self,block_type,channel,block_num,stride=1):
        downsample = None
        if stride !=1 or self.in_channel !=channel*block_type.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel,channel*block_type.expansion,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(channel*block_type.expansion)
            )
        layers = []
        layers.append(block_type(self.in_channel,
                            channel,
                            downsample=downsample,
                            stride=stride,
                            groups=self.groups,
                            width_per_group = self.width_per_group))
        self.in_channel = channel*block_type.expansion
 
        for _ in range(1,block_num):
            layers.append(block_type(self.in_channel,
                                channel,
                                groups=self.groups,
                                width_per_group = self.width_per_group))
        # Sequential:自定义顺序连接成模型，生成网络结构
        return nn.Sequential(*layers)
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # 上述都是静态层，下边是动态层
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.conv2(x)
        if self.linear == True:
            x = torch.flatten(x,1)
            x = self.linearResnet(x)
        
        return x
 
 
# ResNet()中block_type参数对应的位置是BasicBlock或Bottleneck
# ResNet()中blocks_num[0-3]对应[3, 4, 6, 3]，表示残差模块中的残差数
# 34层的resnet
def resnet34():
    # https://download.pytorch.org/models/resnet34-333f7ec4.pth
    return ResNet(BasicBlock, [3, 4, 6, 3])
 
 
# 50层的resnet
def resnet50(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet50-19c8e357.pth
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)
 
 
# 101层的resnet
def resnet101(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet101-5d3b4d8f.pth
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)



In [5]:
learning_rate = 0.001
num_epochs = 300


# 初始化模型、损失函数和优化器
model = resnet34().to(device)

loss_function=nn.MSELoss()

def criterion(output,label,mode = 1):
    if mode == 1:
        return loss_function(output,label)
    
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# 准备 DataLoader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, batch_size=32)

patience_counter = 0
patience_on  = 0
patience = 9
stop_training = 0

best_val_loss = float('inf')

In [6]:

# 热键函数
def on_press(key):
    global patience_on
    global stop_training
    if key.name == 's':#终止训练，储存最好模型
        print("Training stopped. Saving current best model...")
        print(f'best validation loss : {best_val_loss/ len(val_loader)}')
        best_model = model
        best_model.load_state_dict(torch.load('model\Temporary_Model\model_best.pth'))
        # 保存效果最好的模型
        torch.save(best_model.state_dict(), 'model\Temporary_Model\model_best.pth')
        stop_training = 1

    if key.name == 'q': #中途储存当前最好模型，但并不终止训练
        print("Saving current best model to pause1...")
        print(f'best validation loss : {best_val_loss/ len(val_loader)}')
        best_model = model
        best_model.load_state_dict(torch.load('model\Temporary_Model\model_best.pth'))
        # 保存效果最好的模型
        torch.save(best_model.state_dict(), 'model\Temporary_Model\model_pause1.pth')
        
    if key.name == 'w': #中途储存当前最好模型，但并不终止训练
        print("Saving current best model to pause2...")
        print(f'best validation loss : {best_val_loss/ len(val_loader)}')
        best_model = model
        best_model.load_state_dict(torch.load('model\Temporary_Model\model_best.pth'))
        # 保存效果最好的模型
        torch.save(best_model.state_dict(), 'model\Temporary_Model\model_pause2.pth')

    if key.name == 'e': #中途储存当前最好模型，但并不终止训练
        print("Saving current best model to pause3...")
        print(f'best validation loss : {best_val_loss/ len(val_loader)}')
        best_model = model
        best_model.load_state_dict(torch.load('model\Temporary_Model\model_best.pth'))
        # 保存效果最好的模型
        torch.save(best_model.state_dict(), 'model\Temporary_Model\model_pause3.pth')

    if key.name == 'o': #开启early stopping
        patience_on  = 1
        print("early stopping is turned on")

In [9]:
keyboard.on_press(on_press)
# 训练模型

for epoch in range(num_epochs):
    if stop_training:
        break
    model.train()
    train_loss = 0.0
    loss1_total =0.0
    loss2_total =0.0
    loss3_total =0.0
    loss4_total =0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss= criterion(outputs, labels, mode =1)
        loss.backward()
        optimizer.step()
        train_loss += (loss/len(inputs)).item()

    # 验证模型
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)

            val_loss_single = criterion(val_outputs, val_labels, mode =1)
            '''
            loss1 = loss1/len(val_inputs)
            loss2 = loss2/len(val_inputs)
            loss3 = loss3/len(val_inputs)
            loss4 = loss4/len(val_inputs)
            '''
            val_loss += val_loss_single/len(val_inputs)
            '''
            loss1_total += loss1
            loss2_total += loss2
            loss3_total += loss3
            loss4_total += loss4
            '''
    #if val_loss / len(val_loader) <0.002:
        #patience_on = 1
    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 保存效果最好的模型
        torch.save(model.state_dict(), 'model\Temporary_Model\model_best.pth')
    else:
        if patience_on == 1:
            patience_counter += 1
    
    if patience_counter >= patience:
        print(f'在第{epoch+1}个epoch处，Validation loss did not improve for {patience} epochs. Early stopping...')
        print(f'best validation loss : {best_val_loss/ len(val_loader)}')
        break
    if patience_on == 0:
        print(f'Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, best val-loss now : {best_val_loss / len(val_loader)}')
        #print(f'Validation Loss Part: A{loss1_total/ len(val_loader)}, B{loss2_total/ len(val_loader)}, C{loss3_total/ len(val_loader)}, D{loss4_total/ len(val_loader)}')
    else: 
        print(f'Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, earlystopping is on, {patience_counter}steps after last bestloss, best val-loss now : {best_val_loss / len(val_loader)}') 


keyboard.unhook_all()

Epoch 1, Training Loss: 0.12760910334495398, Validation Loss: 0.18723857402801514, earlystopping is on, 0steps after last bestloss, best val-loss now : 0.18723857402801514
Epoch 2, Training Loss: 0.09268295478362304, Validation Loss: 0.174066424369812, earlystopping is on, 0steps after last bestloss, best val-loss now : 0.174066424369812
Epoch 3, Training Loss: 0.07183954664147817, Validation Loss: 0.1585356593132019, earlystopping is on, 0steps after last bestloss, best val-loss now : 0.1585356593132019
Epoch 4, Training Loss: 0.06086513629326454, Validation Loss: 0.1503712683916092, earlystopping is on, 0steps after last bestloss, best val-loss now : 0.1503712683916092
Epoch 5, Training Loss: 0.052030253009154245, Validation Loss: 0.14642742276191711, earlystopping is on, 0steps after last bestloss, best val-loss now : 0.14642742276191711
Epoch 6, Training Loss: 0.044976111788016096, Validation Loss: 0.13814420998096466, earlystopping is on, 0steps after last bestloss, best val-loss 

In [ ]:
# 加载效果最好的模型
best_model = model
best_model.load_state_dict(torch.load('model\Temporary_Model\model_best.pth'))

# 在测试集上评估模型
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1,shuffle = None)

test_loss = 0.0
with torch.no_grad():
    count = 0
    for test_inputs, test_labels in test_loader:
        
        test_outputs = best_model(test_inputs)
        if count == 0:
            
            count = 1
        
        test_loss_single= criterion(test_outputs, test_labels, mode =1)
        test_loss += test_loss_single/len(test_inputs)

        if test_loss_single>0.5 and count == 1:
            count = 2 
            print(test_loss_single)    
        
    print(f'Test Loss: {test_loss / len(test_loader)}')


tensor(3.3656, device='cuda:0')
Test Loss: 2.1963865756988525
